## Cleaning the taxi data 

Using the taxi data and cleaning it to get the foot traffic score

In [ ]:
# import packages
import pandas as pd

# Import package numpy for numeric computing
import numpy as np

# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt

#For showing plots directly in the notebook run the command below
%matplotlib inline

import geopandas as gpd

from shapely.geometry import Point

from haversine import haversine, Unit

from sklearn.preprocessing import MinMaxScaler

Get the Manhatten Zone IDs using taxi_zone_lookup.csv

In [3]:
zone_lookup = pd.read_csv('taxi_zone_lookup.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

In [4]:
zone_lookup.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [5]:
manhattan_zone_ids = zone_lookup[zone_lookup["Borough"] == "Manhattan"]["LocationID"].tolist()

In [ ]:
yellow_taxi_data = pd.read_csv("yellow_tripdata_2023-01.csv")

In [ ]:
yellow_taxi_data["hour"] = pd.to_datetime(yellow_taxi_data["tpep_pickup_datetime"]).dt.hour
# Only Manhattan pickups during daytimeyellow = 
yellow_taxi_data= yellow_taxi_data[
(yellow_taxi_data["PULocationID"].isin(manhattan_zone_ids)) 
& (yellow_taxi_data["hour"].between(7, 20)) & 
(yellow_taxi_data["passenger_count"] > 0)
]

Joining data with TLC zone shapefile

In [ ]:
pickup_counts = yellow_taxi_data.groupby("PULocationID").size().reset_index(name="pickup_count")
zones = gpd.read_file("taxi_zones.shp") # add the taxi zone shape file and after getting the score remove it


In [ ]:
# Merge filtered zones and pickup datazones = zones.merge(zone_lookup, left_on="LocationID", right_on="LocationID")
zones = zones[zones["Borough"] == "Manhattan"]
zones = zones.merge(pickup_counts, left_on="LocationID", right_on="PULocationID", how="left").fillna(0)

In [ ]:
#Normaize Score
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((1, 10))
zones["foot_traffic_score"] = scaler.fit_transform(zones[["pickup_count"]]).round(1)

In [ ]:
#zones.to_file("manhatten_foot_traffic_scores.geojsn", driver=GeoJSON)
zones.to_csv("manhatten_foot_traffic_scores.csv", index=False)